# Exercício - Redes Neurais

### Gustavo Henrique da Silva Costa Araujo
### Matrícula: 20210155800

O objetivo deste exercício é treinar e avaliar uma rede neural para classificar um
conjunto de dados sobre notícias de acordo com suas categorias.

A rede neural deve ser avaliada através do cálculo da acurácia (quantidade de acertos
dividido pela quantidade total de testes).

importando bibliotecas necessárias

In [186]:
import datasets
from datasets import load_dataset, Dataset
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [187]:
dataset = datasets.load_dataset("okite97/news-data")

In [188]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 828
    })
})

separando dataset em treino e teste


In [189]:
train_data = dataset['train']
test_data = dataset['test']

In [190]:
for example in train_data['Title']:
    print(example)

Uefa Opens Proceedings against Barcelona, Juventus and Real Madrid Over European Super League Plan
Amazon Blames Inflation as It Increases Cost of Prime in Europe
Nigeria’s Parliament Passes Amended Electoral Bill
Nigeria: Lagos Governor Tests Positive for Covid-19, Kaduna Governor Self-Isolates
South Africa Calls For Calm as Electoral Reform Protests Continue in Eswatini
Guardiola To Leave Man City When Contract Expires in 2023
Nigeria: Sultan of Sokoto Seeks Removal of Immunity Clause for Politicians
Again, Nigeria Senate Demands Rejig of Nigeria’s Security Architecture
Nigeria: South-East’s Most Important  Demand is Reintegration, Agitations Will End with Igbo Presidency, Says Ex-Senate President Anyim
Premier League Clubs Reject ‘Project Big Picture’
Nigeria: With Advancement in Technology, No One Can Claim Millions of Votes Again, Buhari Says
Old Trafford Modified for 23,500 Socially Distanced Fans
CBN Eyes $200bn from Fresh Non-Oil Exports
Brilliant Brentford Beat Arsenal After 7

criando 3 listas vazias para guardar minhas colunas tratadas e usa-las dentro da função 

In [191]:
title_limp = [] 
categ_limp = []
exc_limp = []



def limpar_texto(dados, dic_limp):
    for index in dados:
        if index is None:
            texto_limpo = ""
        else:
            texto_limpo = index.lower()
            texto_limpo = re.sub(r'[^\w\s]', '', texto_limpo)  # Remova caracteres especiais e pontuações
        dic_limp.append(texto_limpo)  
    
    return dic_limp

In [192]:
limpar_texto(train_data['Title'], title_limp)
limpar_texto(train_data['Excerpt'], exc_limp)
limpar_texto(train_data['Category'], categ_limp)

['sports',
 'business',
 'politics',
 'health',
 'politics',
 'sports',
 'politics',
 'politics',
 'politics',
 'sports',
 'politics',
 'sports',
 'business',
 'sports',
 'tech',
 'business',
 'business',
 'business',
 'politics',
 'health',
 'business',
 'sports',
 'sports',
 'sports',
 'politics',
 'business',
 'sports',
 'entertainment',
 'business',
 'politics',
 'entertainment',
 'politics',
 'politics',
 'politics',
 'tech',
 'health',
 'politics',
 'entertainment',
 'politics',
 'business',
 'politics',
 'business',
 'sports',
 'entertainment',
 'politics',
 'politics',
 'sports',
 'sports',
 'politics',
 'business',
 'business',
 'politics',
 'politics',
 'sports',
 'business',
 'sports',
 'tech',
 'entertainment',
 'sports',
 'tech',
 'business',
 'business',
 'health',
 'business',
 'business',
 'business',
 'business',
 'business',
 'politics',
 'sports',
 'entertainment',
 'business',
 'business',
 'health',
 'health',
 'sports',
 'business',
 'sports',
 'entertainment',
 '

criando minha função para tokenizar palavras e listas

In [193]:
def tokenizar_texto(texto):
    tokens = word_tokenize(texto)
    return tokens

def tokenizar_lista(lista):
    tokens_lista = []  # Crie uma lista para armazenar os tokens

    for texto in lista:
        tokens = tokenizar_texto(texto)
        tokens_lista.append(tokens)  # Adicione os tokens à lista de tokens
    
    return tokens_lista


tokenizando minhas listas já tratadas

In [194]:
title_token = tokenizar_lista(title_limp)
categ_token = tokenizar_lista(categ_limp)
exc_token = tokenizar_lista(exc_limp)

In [195]:
print(title_token[1])

['amazon', 'blames', 'inflation', 'as', 'it', 'increases', 'cost', 'of', 'prime', 'in', 'europe']


criando um novo dataset, com as colunas recebendo os dados tratados e tokenizados

In [196]:
new_data = Dataset.from_dict({"Title": title_token, "Excerpt": train_data['Excerpt'], "Category": train_data['Category']})

In [197]:
new_data

Dataset({
    features: ['Title', 'Excerpt', 'Category'],
    num_rows: 4686
})

utilizando a parte da vetorização para que os dados possam ser usados como entrada para o modelo de classificação. 

In [203]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(new_data['Excerpt'])
y_train = new_data['Category']
X_test = vectorizer.transform(test_data['Excerpt'])
y_test = test_data['Category']

Importando o Classificador Naive Bayes para realizar o treinamento, e logo após calcular sua acurácia.

In [200]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [201]:
classif = MultinomialNB()
classif.fit(X_train, y_train)  # Treinamento com os dados de treinamento

# Faça previsões nos dados de teste
y_pred = classif.predict(X_test)

# Calcule a precisão do modelo nos dados de teste
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo nos dados de teste: {accuracy}")

# Exiba um relatório de classificação nos dados de teste
print(classification_report(y_test, y_pred))

Acurácia do modelo nos dados de teste: 0.7657004830917874
               precision    recall  f1-score   support

     business       0.62      0.96      0.75       224
entertainment       0.94      0.24      0.38        67
       health       0.96      0.45      0.62        97
     politics       0.86      0.88      0.87       190
       sports       0.86      0.96      0.91       199
         tech       0.00      0.00      0.00        51

     accuracy                           0.77       828
    macro avg       0.71      0.58      0.59       828
 weighted avg       0.76      0.77      0.72       828



c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

A acurácia global do modelo nos dados de teste é de aproximadamente 76.57%.

O modelo tem desempenho razoável nas categorias "business", "politics" e "sports", com boas métricas de precisão, recall e pontuação F1.